<a href="https://colab.research.google.com/github/adnan33/120-Data-Science-Interview-Questions/blob/master/Colab%20Notebooks/12.%20Eczema_QA_Experiments/Extract%20biobert%20embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-e99bd034-d37c-c16d-5474-98ba94af5dfb)


In [ ]:
!curl https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip -o uncased_L-12_H-768_A-12.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  389M  100  389M    0     0  60.1M      0  0:00:06  0:00:06 --:--:-- 70.6M


In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:


# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1GJpGjQj6aZPV-EfbiQELpBkvlGtoKiyA' #biobert large
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("biobertL.tar.gz")
file_id='1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD' #biobert base
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("biobertbase.tar.gz")

In [ ]:
!pip -q install transformers

     |████████████████████████████████| 778kB 7.3MB/s 
     |████████████████████████████████| 3.0MB 38.3MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
     |████████████████████████████████| 890kB 37.8MB/s 


In [ ]:
!tar xvzf biobertbase.tar.gz 
!tar xvzf biobertL.tar.gz

In [ ]:
!cp biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/config.json
!cp biobert_large/bert_config_bio_58k_large.json biobert_large/config.json


In [ ]:
import torch
from transformers import BertModel, BertTokenizer
import more_itertools
import numpy as np 

tokenizer = BertTokenizer.from_pretrained("gsarti/biobert-nli")
model = BertModel.from_pretrained("gsarti/biobert-nli", output_hidden_states=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)
model=model.cuda()
def get_embed(text):
  input_ids = torch.tensor(tokenizer.encode(text,add_special_tokens=True,truncation=True, max_length=5000)).unsqueeze(0)  # Batch size 1
  input_ids=input_ids.to(device) 
  outputs = model(input_ids)
  last_hidden_states = outputs[1]  # The last hidden-state is the first element of the output tuple
  last_hidden_states=last_hidden_states.to('cpu')
  return last_hidden_states.detach().numpy()

In [ ]:
import pandas as pd
all_qna=pd.read_csv("/content/all_psoriasis_processed_qna_090720.csv")
allq_text=list(all_qna.iloc[:,0].values)
alla_text=list(all_qna.iloc[:,1].values)
all_qna.describe()

,question,answer
count,33591,33591
unique,4559,33030
top,Why aren't you angry?\n\n[TL;DR: I am bitter. ...,Following
freq,107,43


In [ ]:
def get_embeds(texts):
  embeds=[]
  for i,text in enumerate(texts):
    
    if(len(text)>512):
      print(i)
      w_text=list(more_itertools.windowed(text,n=512,step=512,fillvalue=""))
      j_text=[]
      for w in w_text:
        j_text.append("".join(list(w)))
      s_embeds=[]
      for j in j_text:
        s_embeds.append(get_embed(j))
      embeds.append(np.mean(np.array(s_embeds),axis=0).flatten())
    else:
      embeds.append(get_embed(text).flatten())
  return embeds

In [ ]:
q_embeds=get_embeds(allq_text)
a_embeds=get_embeds(alla_text)

Streaming output truncated to the last 5000 lines.
29677
29678
29679
29680
29681
29682
29683
29684
29685
29686
29687
29688
29689
29690
29691
29692
29693
29694
29695
29723
29724
29725
29726
29727
29728
29729
29730
29731
29732
29733
29734
29735
29736
29737
29738
29739
29740
29741
29742
29743
29744
29745
29746
29747
29748
29749
29750
29751
29752
29753
29754
29755
29756
29757
29758
29759
29760
29761
29762
29763
29764
29765
29766
29767
29768
29769
29770
29771
29772
29773
29774
29775
29776
29777
29778
29779
29780
29781
29782
29783
29784
29785
29786
29787
29788
29789
29790
29791
29792
29793
29794
29795
29796
29797
29798
29799
29800
29801
29802
29803
29804
29805
29806
29807
29808
29809
29810
29811
29812
29813
29814
29815
29816
29817
29818
29819
29820
29821
29822
29823
29824
29829
29830
29831
29832
29833
29834
29835
29836
29837
29838
29839
29840
29841
29842
29853
29854
29855
29856
29857
29858
29859
29860
29861
29862
29863
29864
29865
29866
29867
29868
29869
29870
29871
29872
29873
29874
29875
2

In [ ]:
all_qna["question_bert"]=q_embeds
all_qna["answer_bert"]=q_embeds

In [ ]:
all_qna

,question,answer,question_bert,answer_bert
0,I started this Group in hopes that we may be a...,I can't use chemical cleaners so I use vinegar...,"[0.2513353, -0.042174403, -0.7289861, -0.73366...","[0.2513353, -0.042174403, -0.7289861, -0.73366..."
1,I started this Group in hopes that we may be a...,It is worth a try when we were lil kids mom co...,"[0.2513353, -0.042174403, -0.7289861, -0.73366...","[0.2513353, -0.042174403, -0.7289861, -0.73366..."
2,I started this Group in hopes that we may be a...,2 tablespoons a day is supposed to keep you he...,"[0.2513353, -0.042174403, -0.7289861, -0.73366...","[0.2513353, -0.042174403, -0.7289861, -0.73366..."
3,Hey Friends! I am Kirsten - just wanted to int...,"yes, i have fought the disease naturally., bu...","[0.21648744, -0.051590882, -0.26411286, -0.902...","[0.21648744, -0.051590882, -0.26411286, -0.902..."
4,Hey Friends! I am Kirsten - just wanted to int...,I have fought this dreadful illness for many y...,"[0.21648744, -0.051590882, -0.26411286, -0.902...","[0.21648744, -0.051590882, -0.26411286, -0.902..."
...,...,...,...,...
33586,"New to GP\n\nHi everyone,\n\nNew to Guttate P....",From what I now understand I had flexural/inve...,"[-0.11231208, 0.33283132, -0.82271767, -0.6460...","[-0.11231208, 0.33283132, -0.82271767, -0.6460..."
33587,My Ointment contains Methylprednisolone Acepon...,Methylprednisolone aceponate is a topical ster...,"[0.1974675, 0.039737057, 0.061460048, -0.95352...","[0.1974675, 0.039737057, 0.061460048, -0.95352..."
33588,So I have these rashes..,"Unfortunately, psoriasis cannot be diagnosed o...","[-0.0032412542, 0.27154392, 0.9811811, -0.9999...","[-0.0032412542, 0.27154392, 0.9811811, -0.9999..."
33589,"Anyone tried Ayurvedic herbs (Amla, Shikakai, ...",Hello! It looks like you're asking about scalp...,"[0.074025095, 0.012598759, 0.75807, -0.9913144...","[0.074025095, 0.012598759, 0.75807, -0.9913144..."


In [ ]:
def reshape_embed(embed):
    reshaped_embed = []
    for a in embed:
        a = a.flatten()
        reshaped_embed.append(a)
    return reshaped_embed

In [ ]:
#checking embedding shape for sanity check
#if its 2d then call the reshape embed function
q_embeds[0].shape

(768,)

In [ ]:
#q_embeds = reshape_embed(psoriasis_answer_embeddings)
#a_embeds = reshape_embed(psoriasis_question_embeddings)

In [ ]:
import pickle 
with open('Psoriasis_processed_QAembeddings.pkl','wb') as f:
     pickle.dump(q_embeds, f)
     pickle.dump(a_embeds, f)

In [ ]:
all_qna.to_csv("Psoriasis_processed_QA_with_embedding.csv",index=False)

In [ ]:
!gsutil -m cp /content/*.csv gs://tibot-adnan/qa_files/psoriasis/
!gsutil -m cp /content/*.pkl gs://tibot-adnan/qa_files/psoriasis/

Copying file:///content/all_psoriasis_processed_qna_090720.csv [Content-Type=text/csv]...
Copying file:///content/all_psoriasis_qna_090720.csv [Content-Type=text/csv]...
Copying file:///content/Psoriasis_all_QA_with_embedding.csv [Content-Type=text/csv]...
Copying file:///content/Psoriasis_processed_QA_with_embedding.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloa

In [ ]:

!gsutil -m cp /content/all_QA_with_embedding.csv gs://tibot-adnan/qa_files/all_QA_with_embedding.csv
!gsutil -m cp /content/QAembeddings.pkl gs://tibot-adnan/qa_files/QAembeddings.pkl

Copying file:///content/all_QA_with_embedding.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/731.3 MiB.                                    
Copying file:///content/QAembeddings.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
signific

In [ ]:
all_qna.iloc[4086:4167,:]

,Question,Answer
4162,Hi everyone!\n\nPlease take a few moments of y...,Carissa Maennikka Viaje this may help. I've be...
4163,Hi everyone!\n\nPlease take a few moments of y...,Sarah Parsons this may help your son!
4164,Hi everyone!\n\nPlease take a few moments of y...,Addie Bea this may really help you!
4165,Hi everyone!\n\nPlease take a few moments of y...,Stephanie Locke read this! It will help!
4166,Hi everyone!\n\nPlease take a few moments of y...,Frankie De Soto here is my healing journey wit...
...,...,...
4238,Hi everyone!\n\nPlease take a few moments of y...,So what all do you eat? I feel like that basic...
4239,Hi everyone!\n\nPlease take a few moments of y...,When I started reading this I thought you had ...
4240,Hi everyone!\n\nPlease take a few moments of y...,Debbie Johns I agree with everyone . My son ha...
4241,Hi everyone!\n\nPlease take a few moments of y...,Tayla Jade


In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = './biobert_v1.1_pubmed'
t = text.Transformer(MODEL_NAME, maxlen=512)